Query of a week of 1m binance data of the spot BTC/USDC, SOL/USDC, ETH/USDC

In [ ]:
pair_list = ["BTCUSDT","SOLUSDT","ETHUSDT"]

### dataset 0

In [ ]:
# collect data into dataframe from start date to end date
# https://stackoverflow.com/questions/51358147/fetch-candlestick-kline-data-from-binance-api-using-python-preferably-requests

import requests
import datetime
import pandas as pd
import numpy as np

def get_binance_data_by_requests(ticker, interval, start='2020-01-01 00:00:00', end='2023-07-01 00:00:00',timezone_shift=2):
  """
  interval: str tick interval - 4h/1h/1d ...
  """
  columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
  usecols=['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']
  start = int(datetime.datetime.timestamp(pd.to_datetime(start))*1000)
  end_u = int(datetime.datetime.timestamp(pd.to_datetime(end))*1000)
  df = pd.DataFrame()
  print(f'Downloading {interval} {ticker} ohlc-data ...', end=' ')
  while True:
    url = f'https://www.binance.com/api/v3/klines?symbol={ticker}&interval={interval}&limit=1000&startTime={start}#&endTime={end_u}'
    data = pd.DataFrame(requests.get(url, headers={'Cache-Control': 'no-cache', "Pragma": "no-cache"}).json(), columns=columns, dtype=np.float64)    
    start = int(data.open_time.tolist()[-1])+1
    data.index = [(pd.to_datetime(x, unit='ms') + pd.Timedelta(hours=timezone_shift)).strftime('%Y-%m-%d %H:%M:%S')  for x in data.open_time]
    # data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S')  for x in data.open_time]
    data = data[usecols]
    df = pd.concat([df, data], axis=0)
    if end in data.index.tolist():
      break
  print('Done.')
  df.index = pd.to_datetime(df.index)
  df = df.loc[:end]
  return df#[['open', 'high', 'low', 'close']]

eth = get_binance_data_by_requests(ticker='ETHUSDT', interval='1m', start='2025-04-14 00:00:00', end='2025-04-22 00:00:00')
btc = get_binance_data_by_requests(ticker='BTCUSDT', interval='1m', start='2025-04-14 00:00:00', end='2025-04-22 00:00:00')
sol = get_binance_data_by_requests(ticker='SOLUSDT', interval='1m', start='2025-04-14 00:00:00', end='2025-04-22 00:00:00')


In [ ]:
eth.head(3)

In [ ]:
btc['PAIR'] = "BTCUSDT"
sol['PAIR'] = "SOLUSDT"
eth['PAIR'] = "ETHUSDT"

In [ ]:
out_df = pd.concat([btc,sol,eth])

In [ ]:
# out_df[out_df.PAIR == "BTCUSDT"].head()
# out_df[out_df.PAIR == "ETHUSDT"].head()
# out_df[out_df.PAIR == "SOLUSDT"].head()

Conversion to parquet

In [ ]:
out_df.to_parquet("data/dataset_0.parquet")

## Dataset 1

In [2]:
# collect data into dataframe from start date to end date
# https://stackoverflow.com/questions/51358147/fetch-candlestick-kline-data-from-binance-api-using-python-preferably-requests

import requests
import datetime
import pandas as pd
import numpy as np

def get_binance_data_by_requests(ticker, interval, start='2020-01-01 00:00:00', end='2023-07-01 00:00:00',timezone_shift=2):
  """
  interval: str tick interval - 4h/1h/1d ...
  """
  columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
  usecols = ['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']
  start = int(datetime.datetime.timestamp(pd.to_datetime(start))*1000)
  end_u = int(datetime.datetime.timestamp(pd.to_datetime(end))*1000)
  df = pd.DataFrame()
  print(f'Downloading {interval} {ticker} ohlc-data ...', end=' ')
  while True:
    url = f'https://www.binance.com/api/v3/klines?symbol={ticker}&interval={interval}&limit=1000&startTime={start}#&endTime={end_u}'
    data = pd.DataFrame(requests.get(url, headers={'Cache-Control': 'no-cache', "Pragma": "no-cache"}).json(), columns=columns, dtype=np.float64)    
    start = int(data.open_time.tolist()[-1])+1
    data.index = [(pd.to_datetime(x, unit='ms') + pd.Timedelta(hours=timezone_shift)).strftime('%Y-%m-%d %H:%M:%S')  for x in data.open_time]
    # data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S')  for x in data.open_time]
    data = data[usecols]
    df = pd.concat([df, data], axis=0)
    if end in data.index.tolist():
      break
  print('Done.')
  df.index = pd.to_datetime(df.index)
  df = df.loc[:end]
  return df#[['open', 'high', 'low', 'close']]

eth = get_binance_data_by_requests(ticker='ETHUSDT', interval='1s', start='2025-04-14 00:00:00', end='2025-04-17 00:00:00')
btc = get_binance_data_by_requests(ticker='BTCUSDT', interval='1s', start='2025-04-14 00:00:00', end='2025-04-17 00:00:00')
sol = get_binance_data_by_requests(ticker='SOLUSDT', interval='1s', start='2025-04-14 00:00:00', end='2025-04-17 00:00:00')

In [3]:
btc['PAIR'] = "BTCUSDT"
sol['PAIR'] = "SOLUSDT"
eth['PAIR'] = "ETHUSDT"

out_df = pd.concat([btc,sol,eth])

out_df.to_parquet("data/dataset_1.parquet")